In [6]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pypianoroll
import random

import tensorflow.keras as keras
import midi

In [4]:
y = np.load('pypianorollSamples.npy')

In [7]:
music = y[2].reshape(2048,72)
erased_music, mask = midi.erase_notes(music, 0.25, 72)

In [8]:
music.transpose().shape

(72, 2048)

In [9]:
input_data = np.expand_dims(np.stack([erased_music, mask], axis=2), axis=0)
output_data = np.expand_dims(music.transpose(), axis=0)

In [10]:
input_data.shape

(1, 72, 2048, 2)

In [11]:
output_data.shape

(1, 72, 2048)

In [ ]:
_, rows, cols, _ = input_data.shape
model = keras.models.Sequential([
    keras.layers.Conv2D(5, 3, activation="relu", padding="same", input_shape=(rows, cols, 2)),
    keras.layers.Conv2D(1, 3, activation="sigmoid", padding="same"),
    keras.layers.Reshape((rows, cols))
])

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam")

history = model.fit(input_data, output_data, epochs=500)

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
def mask_random_notes(music):
    fake_input = np.zeros(music.shape + (2,))
    num_notes_to_mask = 30
    for _ in range(num_notes_to_mask):
        note_to_mask = random.randint(0, input_data.shape[2] - 1)
        fake_input[0, :, note_to_mask, 0] = 0
        fake_input[0, :, note_to_mask, 1] = 1

    return fake_input

def make_music_iteration(fake_input):
    new_music = model(fake_input)
    rand_vals = np.random.uniform(size=new_music.shape)
    new_music = new_music > rand_vals
    new_music = new_music.numpy().astype('float')
    return new_music

In [ ]:
new_music = np.zeros(input_data.shape[:-1])
for _ in range(10):
    fake_input = mask_random_notes(new_music)
    new_music = make_music_iteration(fake_input)


In [ ]:
new_music

In [ ]:
plt.figure(figsize=(15,12))
plt.imshow(new_music[0, :, :512])

In [ ]:
input_data[0,:,:10,0]